In [126]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use('fivethirtyeight')

#preprocessing
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag


In [127]:
import DataCleaning_reviews_5

In [128]:
df5 = DataCleaning_reviews_5.data_clean('../reviews_Grocery_and_Gourmet_Food_5.json.gz')

302508
{'reviewerID': 'A1VEELTKS8NLZB', 'asin': '616719923X', 'reviewerName': 'Amazon Customer', 'helpful': [0, 0], 'reviewText': 'Just another flavor of Kit Kat but the taste is unique and a bit different.  The only thing that is bothersome is the price.  I thought it was a bit expensive....', 'overall': 4.0, 'summary': 'Good Taste', 'unixReviewTime': 1370044800, 'reviewTime': '06 1, 2013'}
302508


In [129]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   reviewerID     151254 non-null  object 
 1   asin           151254 non-null  object 
 2   reviewerName   149761 non-null  object 
 3   reviewText     151254 non-null  object 
 4   helpful_votes  151254 non-null  object 
 5   rating         151254 non-null  float64
 6   reviewTitle    151254 non-null  object 
dtypes: float64(1), object(6)
memory usage: 9.2+ MB


In [130]:
df = df5[['reviewText','reviewTitle', 'asin']]

In [131]:
df.head()

,reviewText,reviewTitle,asin
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X
2,Really good. Great gift for any fan of green t...,Yum!,616719923X
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X


In [132]:
# check for null values
print(df['reviewText'].isnull().sum())
df['reviewTitle'].isnull().sum()  # no null values.

print(df.shape)
df.head()

0
(151254, 3)


,reviewText,reviewTitle,asin
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X
2,Really good. Great gift for any fan of green t...,Yum!,616719923X
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X


In [133]:
df["all_text"] = df["reviewText"] + ' ' + df["reviewTitle"]


<ipython-input-133-fab1430bca7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["all_text"] = df["reviewText"] + ' ' + df["reviewTitle"]


In [2]:
df.head()

NameError: name 'df' is not defined

In [135]:
stemmer_porter=PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
import string  
snowball_stem = SnowballStemmer(language='english')
s_words = set(stopwords.words('english'))
punc = set(string.punctuation)
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
df['cleanText1']= df['all_text'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
df['cleanText2'] = df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
df['cleanText3'] = df['cleanText2'].apply(is_only_alpha)
df['cleanText4'] = df['cleanText3'].apply(word_tokenize)
df['cleanText5'] = df['cleanText4'].apply(lambda words: [word.lower() for word in words])
df['cleanText6'] = df['cleanText5'].apply(stemmers)
df['cleanText7'] = df['cleanText6'].apply(lambda words: " ".join(words))
df = df.drop(['cleanText1','cleanText2','cleanText3','cleanText4','cleanText5','cleanText6'],axis=1)

<ipython-input-135-9b8d129e3e2b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleanText1']= df['all_text'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
<ipython-input-135-9b8d129e3e2b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleanText2'] = df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))


In [136]:
vectorizer = TfidfVectorizer(max_features=10_000, lowercase=False)
X = vectorizer.fit_transform(df['cleanText7'])

In [137]:
X[10:,10:].todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.20658839, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [138]:
# X is the tf-idf of the reviews 
# y is the target or the inputed review from the user
user_input = "Without a doubt, pizza is the greatest food that has ever been developed. It has all the major food groups, an nice mix of carbohydrates, protein, calcium, and can be loaded down the vegetables. Some people don't like the gluten in the crust, so a gluten-free option is like hitting the trifecta."
user_df = pd.DataFrame({'user1': [user_input]})
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
user_df['cleanText1']= user_df['user1'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
user_df['cleanText2'] = user_df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
user_df['cleanText3'] = user_df['cleanText2'].apply(is_only_alpha)
user_df['cleanText4'] = user_df['cleanText3'].apply(word_tokenize)
user_df['cleanText5'] = user_df['cleanText4'].apply(lambda words: [word.lower() for word in words])
user_df['cleanText6'] = user_df['cleanText5'].apply(stemmers)
user_df['cleanText7'] = user_df['cleanText6'].apply(lambda words: " ".join(words))
#print(user_df['cleanText7'])
user_df = df.drop(['cleanText1','cleanText2','cleanText3','cleanText4','cleanText5','cleanText6'],axis=1)
y = vectorizer.transform([user_df['cleanText7'][0]])

In [139]:
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(y,X)
sim_sorted= np.argsort(sim_matrix)[::-1] # idx of lowest value to highest value but with ::-1 its decending
sim_sorted

array([[75626, 87463, 36845, ..., 38110,  2959, 15071]])

In [140]:
df['cleanText7'][sim_sorted[0][-2]]

'these realli great tomato i hate metal tast can tomato tast like chanc leach tomato can t healthi these rich flavor wateri great sauc virgin oliv oil fresh basil garlic s p cook pasta firm finish cook sauc simpl delici love tomato wish i could find local least i get amazon ever without amazon no tinni metal tast'

In [1]:
item_asin_top1 = df.iloc[sim_sorted[0][-8]].loc['asin']
item_asin_top1
item_asin_top10 = []
for i in range(10):
    item_asin_top10.append(df.iloc[sim_sorted[0][-i]].loc['asin'])
top10_url = []
for asin in item_asin_top10:
    url = "http://www.amazon.com/dp/"+asin
    top10_url.append(url)
top10_url

# now I have to find if one of these is glutine free, 
#organic or keto and stuff like that. also try to find if I can 
# find broken links by cleaining up the reviews with only ones in meta
# berfore that I have to use meta to find the ones that have organic or glutine free
# then clean up reviews using only the ones that i have in meta after the fact
# and then clean up the recomendations so it doesnt rocomend only the same item like 
# pop tars and tomato sause

NameError: name 'df' is not defined

In [142]:
import DataCleaning_meta

In [ ]:
df_meta = DataCleaning_meta.data_clean_meta("../fmeta_Grocery_and_Gourmet_Food.json.gz")

In [ ]:
df_meta[df_meta['asin'] == item_asin_top1]

,reviewText,reviewTitle,asin,all_text
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X,Just another flavor of Kit Kat but the taste i...
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X,I bought this on impulse and it comes from Jap...
2,Really good. Great gift for any fan of green t...,Yum!,616719923X,Really good. Great gift for any fan of green t...
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X,"I had never had it before, was curious to see ..."
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X,I've been looking forward to trying these afte...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   reviewText   151254 non-null  object
 1   reviewTitle  151254 non-null  object
 2   asin         151254 non-null  object
 3   all_text     151254 non-null  object
dtypes: object(4)
memory usage: 5.8+ MB


'Dishes cooked with nutrela are very tasty and also have natural nutritive goodness. Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition. Nutrela High Protein Soya Chunks 200g (Pack of 6) Grocery & Gourmet Food Canned, Jarred & Packaged Foods Packaged Meals & Side Dishes Indian Dishes'

<ipython-input-26-7fd8966e5400>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['all_text'] = df_meta_life['description'] + ' ' + df_meta_life['name']
<ipython-input-26-7fd8966e5400>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['all_text'] = df_meta_life['all_text'] + ' ' + df_meta_life['categories']


KeyError: 4

<ipython-input-52-14342b4481e2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['cleanText1']= df_meta_life['all_text'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
<ipython-input-52-14342b4481e2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['cleanText2'] = df_meta_life['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
<ipython-input-52-14342b4481e2>:7: SettingWi

In [53]:
df_meta_life['cleanText7'].iloc[0]

'beemster gouda chees age month statement regard dietari supplement evalu fda intend diagnos treat cure prevent diseas health condit beemster gouda age month app lb groceri gourmet food dairi chees egg chees gouda'

In [ ]:
# def is_only_alpha(text):
#     return " ".join([word for word in text.split() if word.isalpha()])
# def stemmers(llst):
#     return [stemmer_porter.stem(words) for words in llst]
# df_meta_life['cleanText1']= df_meta_life['all_text'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
# df_meta_life['cleanText2'] = df_meta_life['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
# df_meta_life['cleanText3'] = df_meta_life['cleanText2'].apply(is_only_alpha)
# df_meta_life['cleanText4'] = df_meta_life['cleanText3'].apply(word_tokenize)
# df_meta_life['cleanText5'] = df_meta_life['cleanText4'].apply(lambda words: [word.lower() for word in words])

# df_meta_life['cleanText6'] = df_meta_life['cleanText5'].apply(lambda words: " ".join(words))

In [54]:
df_meta_life.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261845 entries, 0 to 287050
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   description    261845 non-null  object
 1   brand          261845 non-null  object
 2   rank           261845 non-null  object
 3   price          261845 non-null  object
 4   asin           261845 non-null  object
 5   name           261845 non-null  object
 6   features       261845 non-null  object
 7   categories     261845 non-null  object
 8   main_category  261845 non-null  object
 9   all_text       261845 non-null  object
 10  cleanText1     261845 non-null  object
 11  cleanText2     261845 non-null  object
 12  cleanText3     261845 non-null  object
 13  cleanText4     261845 non-null  object
 14  cleanText5     261845 non-null  object
 15  cleanText6     261845 non-null  object
 16  cleanText7     261845 non-null  object
dtypes: object(17)
memory usage: 46.0+ MB


In [114]:
lifestyles = ['vegetarian?','paleo?','keto?', 'vegan?','lowsugar?',"low sugar",'gluten free', "gluten",'low gluten',"pescaterian",'low fat','fat free','free fat',"organic?", "gmo",'sugar?' ,'subsitutes','indian', "Kosher", "halal", 'Ovo lacto','lacto?']
df_meta_filtered = df_meta_life[df_meta_life['cleanText7'].str.contains('|'.join(lifestyles))]


In [115]:
df_meta_filtered.head()

,description,brand,rank,price,asin,name,features,categories,main_category,all_text,cleanText1,cleanText2,cleanText3,cleanText4,cleanText5,cleanText6,cleanText7
1,"Shipped from UK, please allow 10 to 21 busines...",,"315,867 in Grocery & Gourmet Food (",,0853347867,Trim Healthy Mama Xylitol,[],Grocery & Gourmet Food Cooking & Baking Sugar ...,Grocery,"Shipped from UK, please allow 10 to 21 busines...","Shipped UK, please allow 10 21 business days a...",Shipped UK please allow 10 21 business days a...,Shipped UK please allow business days arrival ...,"[Shipped, UK, please, allow, business, days, a...","[shipped, uk, please, allow, business, days, a...","[ship, uk, pleas, allow, busi, day, arriv, ex,...",ship uk pleas allow busi day arriv ex lib pub ...
13,Dishes cooked with nutrela are very tasty and ...,Nutrela,"195,125 in Grocery & Gourmet Food (",$24.99,5236363640,Nutrela High Protein Soya Chunks 200g (Pack of 6),[],"Grocery & Gourmet Food Canned, Jarred & Packag...",Grocery,Dishes cooked with nutrela are very tasty and ...,Dishes cooked nutrela tasty also natural nutri...,Dishes cooked nutrela tasty also natural nutri...,Dishes cooked nutrela tasty also natural nutri...,"[Dishes, cooked, nutrela, tasty, also, natural...","[dishes, cooked, nutrela, tasty, also, natural...","[dish, cook, nutrela, tasti, also, natur, nutr...",dish cook nutrela tasti also natur nutrit good...
14,Haldiram&apos;s Bhelpuri - A Mild Spicy Blend ...,Haldirams,"72,407 in Grocery & Gourmet Food (",$18.99,541255556X,Haldiram's Bhelpuri - A Mild Spicy Blend of Cr...,[],"Grocery & Gourmet Food Canned, Jarred & Packag...",Grocery,Haldiram&apos;s Bhelpuri - A Mild Spicy Blend ...,Haldiram&apos;s Bhelpuri - A Mild Spicy Blend ...,Haldiram apos s Bhelpuri A Mild Spicy Blend ...,Haldiram apos s Bhelpuri A Mild Spicy Blend Cr...,"[Haldiram, apos, s, Bhelpuri, A, Mild, Spicy, ...","[haldiram, apos, s, bhelpuri, a, mild, spicy, ...","[haldiram, apo, s, bhelpuri, a, mild, spici, b...",haldiram apo s bhelpuri a mild spici blend cri...
16,Kashmiri Mirch is a special blend of medium ho...,MDH,"329,103 in Grocery & Gourmet Food (",,5478541265,MDH Kashmiri Mirch 100g,[],"Grocery & Gourmet Food Canned, Jarred & Packag...",Grocery,Kashmiri Mirch is a special blend of medium ho...,Kashmiri Mirch special blend medium hot qualit...,Kashmiri Mirch special blend medium hot qualit...,Kashmiri Mirch special blend medium hot qualit...,"[Kashmiri, Mirch, special, blend, medium, hot,...","[kashmiri, mirch, special, blend, medium, hot,...","[kashmiri, mirch, special, blend, medium, hot,...",kashmiri mirch special blend medium hot qualit...
21,Gur Revdi is made by mixing Sesame seeds with ...,Swad,"173,342 in Grocery & Gourmet Food (",$9.28,618205610X,GUR REWDI - Jaggery Candy with Sesame Seeds 7o...,[],Grocery & Gourmet Food Cooking & Baking Nuts &...,Grocery,Gur Revdi is made by mixing Sesame seeds with ...,Gur Revdi made mixing Sesame seeds jaggery glu...,Gur Revdi made mixing Sesame seeds jaggery glu...,Gur Revdi made mixing Sesame seeds jaggery glu...,"[Gur, Revdi, made, mixing, Sesame, seeds, jagg...","[gur, revdi, made, mixing, sesame, seeds, jagg...","[gur, revdi, made, mix, sesam, seed, jaggeri, ...",gur revdi made mix sesam seed jaggeri glucos t...


In [116]:
df_meta_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96775 entries, 1 to 287050
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   description    96775 non-null  object
 1   brand          96775 non-null  object
 2   rank           96775 non-null  object
 3   price          96775 non-null  object
 4   asin           96775 non-null  object
 5   name           96775 non-null  object
 6   features       96775 non-null  object
 7   categories     96775 non-null  object
 8   main_category  96775 non-null  object
 9   all_text       96775 non-null  object
 10  cleanText1     96775 non-null  object
 11  cleanText2     96775 non-null  object
 12  cleanText3     96775 non-null  object
 13  cleanText4     96775 non-null  object
 14  cleanText5     96775 non-null  object
 15  cleanText6     96775 non-null  object
 16  cleanText7     96775 non-null  object
dtypes: object(17)
memory usage: 13.3+ MB


In [117]:
df_meta_filtered.iloc[100]

description      A harmony of chocolates and flowers which make...
brand                                       Helen Grace Chocolates
rank                           903,053 in Grocery & Gourmet Food (
price                                                             
asin                                                    B0000D89R5
name             Helen Grace Chocolates, Long-Stemmed Truffle R...
features                                                        []
categories       Grocery & Gourmet Food Food & Beverage Gifts C...
main_category                                              Grocery
all_text         A harmony of chocolates and flowers which make...
cleanText1       A harmony chocolates flowers makes truly memor...
cleanText2       A harmony chocolates flowers makes truly memor...
cleanText3       A harmony chocolates flowers makes truly memor...
cleanText4       [A, harmony, chocolates, flowers, makes, truly...
cleanText5       [a, harmony, chocolates, flowers, makes, trul

In [119]:
#df_meta_filtered = df_meta_filtered.drop(['cleanText1', 'cleanText2',
#       'cleanText3', 'cleanText4', 'cleanText5', 'cleanText6'],axis=1)

In [121]:
#df_meta_filtered.to_csv('meta_filtred.csv')

In [122]:
#df5 = pd.read_csv('meta_filtred.csv')

In [125]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96775 entries, 0 to 96774
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     96775 non-null  int64 
 1   description    93021 non-null  object
 2   brand          95510 non-null  object
 3   rank           96775 non-null  object
 4   price          44878 non-null  object
 5   asin           96775 non-null  object
 6   name           96775 non-null  object
 7   features       96775 non-null  object
 8   categories     96775 non-null  object
 9   main_category  96775 non-null  object
 10  all_text       96775 non-null  object
 11  cleanText7     96775 non-null  object
dtypes: int64(1), object(11)
memory usage: 8.9+ MB


In [124]:
#df_meta_life.to_csv('meta__all_text.csv')

In [ ]:
m = d['asin'].isin(df_meta_['asin'])
df2 = df2[m]